In [10]:
import numpy as np
import polars as pl
from polars import col, lit

In [12]:
np.random.seed(12)

df = pl.DataFrame(
    {
        "nrs": [1, 2, 3, None, 5],
        "names": ["foo", "ham", "spam", "egg", None],
        "random": np.random.rand(5),
        "groups": ["A", "A", "B", "C", "B"],
    }
)
df

nrs,names,random,groups
i64,str,f64,str
1,"""foo""",0.154163,"""A"""
2,"""ham""",0.74005,"""A"""
3,"""spam""",0.263315,"""B"""
null,"""egg""",0.533739,"""C"""
5,null,0.014575,"""B"""


## Polars expressions

Below is a Polars expression:

In [30]:
df.select([
    col("random").sort().head(2),
    col("nrs").filter(col("names").is_in(["foo", "ham", "spam"])).sum()  # this will get broadcast across rows because it's a scalar
])

random,nrs
f64,i64
0.014575,6
0.154163,6


All expressions are run in parallel

### Unique values

In [33]:
df.select([
    col("names").n_unique().alias("unique_names_1"),
    col("names").unique().count().alias("unique_names_2"),  # Polars doesn't like duplicate column names
])

unique_names_1,unique_names_2
u32,u32
5,5
